# Amazon AthenaとAWS Glue Catalogを利用してS3内のデータセットをクエリする

このノートブックでは、Glue Catalog（Hive Metastore）にAthenaテーブルを作成し、S3のデータセットファイルに対してクエリできるようにします。
S3の `Amazon Customer Reviews Dataset` を基に、Glue Catalogにテーブルを作成します。
データに対応するカラムを定義し、データのフォーマットを指定し、Amazon S3内のファイルの場所を指定します。

<img src="img/athena_register_parquet.png" width="60%" align="left">

In [ ]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

In [ ]:
from pyathena import connect
import pandas as pd

In [ ]:
%store -r setup_dependencies_passed

In [ ]:
try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
print(setup_dependencies_passed)

In [ ]:
if not setup_dependencies_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

# Parquetデータに対してAthenaテーブルを作成

#### データセットのカラム

- `marketplace`: 二文字の国コード（今回はすべて「US」）。
- `customer_id`: それぞれの書き手のレビュー集約に使われるランダムID。
- `review_id`: レビューのユニークID。
- `product_id`: Amazon標準識別番号（ASIN）。`http://www.amazon.com/dp/<ASIN>` が商品の詳細ページへのリンクとなる。
- `product_parent`: ASINの親。ひとつの親に対して複数のASINが存在しうる（同じ商品の色違いやフォーマット違いなど）。
- `product_title`: 商品のタイトル表記。
- `product_category`: レビューのグループ化に使う大まかな商品カテゴリー（このケースでは「デジタルビデオ」）
- `star_rating`: レビューの星評価（1〜5）。
- `helpful_votes`: レビューへの「役に立った」投票の個数。
- `total_votes`: レビューへの全投票数。
- `vine`: レビューが[Vine](https://www.amazon.com/gp/vine/help)先取りプログラムの一環で書かれたか否か。
- `verified_purchase`: レビューが検証済みの購入対するものか否か。
- `review_headline`: レビューそのもののタイトル。
- `review_body`: レビュー本体のテキスト。
- `review_date`: レビューが投稿された日付。

In [ ]:
s3_staging_dir = "s3://{}/athena/query-cache".format(bucket)

In [ ]:
database_name = "default"
table_name = "amazon_reviews_parquet"

In [ ]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [ ]:
# 実行するSQL文
statement = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
      marketplace string, 
      customer_id string, 
      review_id string, 
      product_id string, 
      product_parent string, 
      product_title string, 
      star_rating int, 
      helpful_votes int, 
      total_votes int, 
      vine string, 
      verified_purchase string, 
      review_headline string, 
      review_body string, 
      review_date bigint, 
      year int)
    PARTITIONED BY (product_category string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
      's3://amazon-reviews-pds/parquet/'
""".format(
    database_name, table_name
)

print(statement)

pd.read_sql(statement, conn)

# パーティションの分析と構築

In [ ]:
statement = """
    MSCK REPAIR TABLE {}.{}
""".format(
    database_name, table_name
)

print(statement)

pd.read_sql(statement, conn)

# テーブルが問題なく作成されたことをチェック

In [ ]:
statement = "SHOW TABLES in {}".format(database_name)

df_tables = pd.read_sql(statement, conn)
df_tables

# サンプルクエリを実行

In [ ]:
product_category = "Digital_Software"

statement = """
    SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100
""".format(
    database_name, table_name, product_category
)

print(statement)

In [ ]:
df = pd.read_sql(statement, conn)
df.head(5)

In [ ]:
if df.empty:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA.")
    print("LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.             ")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

In [ ]:
if "amazon_reviews_parquet" in df_tables.values and not df.empty:
    ingest_create_athena_table_parquet_passed = True

In [ ]:
%store ingest_create_athena_table_parquet_passed

# Glue Catalogで新しく作成したAthenaテーブルをレビュー

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#table:catalog={};name=amazon_reviews_parquet;namespace=default">AWS Glue Catalog</a></b>'.format(
            region, account_id
        )
    )
)

# 以降のノートブックのために変数を保存

In [ ]:
%store

# リソースを解放

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>